In [93]:
import os

import numpy as np
import pykeen
from matplotlib import pyplot as plt

from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory

from numbers import Complex
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

from pykeen.models import TransE
from torch.optim import Adam
from pykeen.training import SLCWATrainingLoop
from pykeen.evaluation import RankBasedEvaluator


%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [2]:
def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=False)
    from IPython.display import FileLink
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [140]:
import pandas as pd
from os.path import exists

from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from pykeen.datasets.nations import NATIONS_TRAIN_PATH
# dataset = "/eos/jeodpp/home/users/chenina/data/works1k.ttl.nt"
base_dir = "/home/nasredine/dev/python/graph_embedding/pyKEEN/data/"
dataset = base_dir + "graph.nt.csv"



In [130]:
# cleaning data
# split one column row to three columns
# df = pd.read_csv(dataset, sep='\t', header=None, names=['s', 'p', 'o'])
# display 10 row that have s start with "cellar:"
# dx = df[df['s'].str.startswith('cellar:')]
# split dx to three columns using 5 spaces as separator
# options
# pd.options.display.max_colwidth = 200
# display full content of fitst row
# dx.iloc[0]
# split one column row to three columns
# expand first column to three columns
# dy = dx.s.str.split('     ', expand=True)
# dy.to_csv(base_dir + "graph0_cleaned.csv", index=False, header=False, sep='\t')

pandas.core.frame.DataFrame

In [141]:
tf = TriplesFactory.from_path(dataset)


In [142]:
training_path = base_dir +dataset+ "_training.pt"
testing_path = base_dir + dataset+ "_testing.pt"

if not exists(training_path) or not exists(testing_path):
    training  , testing = tf.split()
    training.to_path_binary(training_path)
    testing.to_path_binary(testing_path)
else :
    training = TriplesFactory.from_path_binary(training_path)
    testing = TriplesFactory.from_path_binary(testing_path)

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [129210, 47160]
INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=60188, num_relations=8, num_triples=188640, inverse_triples=False, path="/home/nasredine/dev/python/graph_embedding/pyKEEN/data/graph.nt.csv") to file:///home/nasredine/dev/python/graph_embedding/pyKEEN/data/home/nasredine/dev/python/graph_embedding/pyKEEN/data/graph.nt.csv_training.pt
INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=60188, num_relations=8, num_triples=47160, inverse_triples=False, path="/home/nasredine/dev/python/graph_embedding/pyKEEN/data/graph.nt.csv") to file:///home/nasredine/dev/python/graph_embedding/pyKEEN/data/home/nasredine/dev/python/graph_embedding/pyKEEN/data/graph.nt.csv_testing.pt


pykeen.triples.triples_factory.TriplesFactory

In [143]:
def train():
    
    result = pipeline(
        training=training,
        testing=testing,
        model='TransE',
        epochs=5,  # 228 short epochs for testing - you should go higher
    )
    model = result.model
    result.save_to_directory('data/test_unstratified_transe')

In [40]:
# result.plot()


bool

In [145]:
import torch

#  if pre trained model exists, load it
model_path = base_dir +dataset+ "_model.pkl"

if exists(model_path):
    print("loading existing model")
    result = torch.load(model_path)
    model = result
else :
    train()


INFO:pykeen.pipeline.api:Using device: None
/home/nasredine/.local/lib/python3.10/site-packages/pykeen/nn/representation.py:372: UserWarning: Directly use Embedding.max_id instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.max_id instead of num_embeddings.")


Training epochs on cpu:   0%|          | 0/5 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/737 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/737 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/737 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/737 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/737 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.


Evaluating on cpu:   0%|          | 0.00/47.2k [00:00<?, ?triple/s]

KeyboardInterrupt: 

In [47]:
testing.triples

pykeen.triples.triples_factory.TriplesFactory

In [53]:
testing.entities_to_ids(["<http://publications.europa.eu/resource/cellar/059e98ea-049c-49e2-a348-6aaa5157136b>"])

[1125]

In [ ]:
testing.triples

In [91]:
from pykeen.models.predict import get_tail_prediction_df

def predict_eurovoc(head_label,relation_label,triples_factory,model ): 
    print(head_label)
    df = get_tail_prediction_df(
    model=model,
    head_label= head_label,
    relation_label= relation_label,
    triples_factory=triples_factory,
    add_novelties=False,
    )
    
    return df

In [92]:
# training_triples_factory = result.training
head_label="<http://publications.europa.eu/resource/cellar/059e98ea-049c-49e2-a348-6aaa5157136b>"
relation_label="<http://publications.europa.eu/ontology/cdm#work_is_about_concept_eurovoc>"
df = predict_eurovoc(head_label, relation_label,training,model)
df

<http://publications.europa.eu/resource/cellar/059e98ea-049c-49e2-a348-6aaa5157136b>


,tail_id,tail_label,score
588,588,<http://eurovoc.europa.eu/5589>,-6.211856
553,553,<http://eurovoc.europa.eu/4360>,-6.340346
572,572,<http://eurovoc.europa.eu/5043>,-6.341881
1125,1125,<http://publications.europa.eu/resource/cellar...,-6.392386
1560,1560,ep:P7_QE(2010)4660^^<http://www.w3.org/2001/XM...,-6.751966
...,...,...,...
274,274,2021-07-05T17:15:21.179+02:00^^<http://www.w3....,-10.841079
101,101,1996-10-02^^<http://www.w3.org/2001/XMLSchema#...,-10.842767
1055,1055,<http://publications.europa.eu/resource/author...,-10.851442
339,339,32015R0768^^<http://www.w3.org/2001/XMLSchema#...,-10.881003


In [ ]:
df.head(20)

In [ ]:
pd.options.display.max_colwidth= 100
pd.options.display.max_rows=  500

df.head(500)

In [ ]:
evaluator = RankBasedEvaluator()

# Get triples to test
mapped_triples = testing.mapped_triples


# Evaluate
results = evaluator.evaluate(
    model=model,
    mapped_triples=mapped_triples,
    batch_size=1024,
    additional_filter_triples=[
        training.mapped_triples,
        dataset.validation.mapped_triples,
    ],
)


model = Complex()

# Evaluate
results = evaluator.evaluate(
    model=model,
    mapped_triples=mapped_triples,
    batch_size=1024,
    additional_filter_triples=[
        training.mapped_triples,
        validation.mapped_triples,
    ],
)
print(results)


In [ ]:
pykeen.env()

In [ ]:
os.makedirs('results', exist_ok=True)

triples = '''
Brussels	locatedIn	Belgium
Belgium	partOf	EU
EU	hasCapital	Brussels
'''.strip()

triples = np.array([triple.split('\t') for triple in triples.split('\n')])
tf = TriplesFactory.from_labeled_triples(triples=triples)

In [ ]:
results = pipeline(
    training=tf,
    testing=tf,
    model = 'TransE',
    model_kwargs=dict(embedding_dim=2),
    training_kwargs=dict(use_tqdm_batch=False),
    evaluation_kwargs=dict(use_tqdm=False),

)
results.plot(er_kwargs=dict(plot_relations=True))
plt.savefig('results/toy_1.png', dpi=300)

In [ ]:
results = pipeline(
    training=tf,
    testing=tf,
    model = 'TransE',
    model_kwargs=dict(embedding_dim=2),
    optimizer_kwargs=dict(lr=1.0e-1),
    training_kwargs=dict(num_epochs=128, use_tqdm_batch=False),
    evaluation_kwargs=dict(use_tqdm=False),
 
)
results.plot(er_kwargs=dict(plot_relations=True))
plt.savefig('results/toy_2.png', dpi=300)

In [ ]:
toy_results = pipeline(
    training=tf,
    testing=tf,
    model='TransE',
    loss='softplus',
    model_kwargs=dict(embedding_dim=2),
    optimizer_kwargs=dict(lr=1.0e-1),
    training_kwargs=dict(num_epochs=128, use_tqdm_batch=True),
    evaluation_kwargs=dict(use_tqdm=True),
    random_seed=2501062933,

)

In [ ]:
toy_results.plot(er_kwargs=dict(plot_relations=True))
plt.savefig('results/toy_3.png', dpi=300)

In [ ]:
nations_results = pipeline(
    dataset='Nations',
    model='TransE',
    model_kwargs=dict(embedding_dim=8),
    optimizer_kwargs=dict(lr=1.0e-1),
    training_kwargs=dict(num_epochs=80, use_tqdm_batch=False),
    evaluation_kwargs=dict(use_tqdm=False),
   
)

In [ ]:
nations_results.plot(er_kwargs=dict(plot_relations=True))


In [ ]:
# Filter the ER plot down to a specific set of entities and relations
nations_results.plot_er(
    relations={'treaties'},
    apply_limits=False,
    plot_relations=True,
);

In [ ]:
nations_results.metric_results.to_df()


In [ ]:
tf = nations_results.training
tf

In [ ]:
tf.entity_word_cloud()
